# quickstart
介绍机器学习中常见的 api.

## 1.处理数据
pytorch 提供了 2 个包来处理数据：torch.utils.data.DataLoader 和 torch.utils.data.Dataset。
Dataset 存储样本及相应的标签。 DataLoader 围绕 Dataset 包装了一个可迭代对象.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# pytorch提供特定领域的库，例如 TorchText, TorchVision, TorchAudio,所有这些都包含数据集。

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:16<00:00, 1586126.36it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 56356.63it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:04<00:00, 1076710.50it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 9243269.26it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
# 将 Dataset 作为参数传递给 DataLoader 。在数据集上包装了一个可迭代对象，并支持自动批处理、采样、洗牌和多进程数据加载。
# 定义批量大小为 64，即数据加载器迭代中的每个元素将返回一批 64 个特征和标签。

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 2.创建模型

为了在 PyTorch 中定义神经网络，我们创建一个继承自nn.Module的类。我们在__init__函数中定义网络层，并在前forward函数中指定数据如何通过网络。为了加速神经网络中的操作，我们将其转移到 GPU 或 MPS（如果可用）。

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 3.优化模型参数
为了训练模型，需要一个损失函数和一个优化器。

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
# 在单个训练循环中，模型对训练数据集（批量输入）进行预测，并反向传播预测误差以调整模型的参数。
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
# 还根据测试数据集检查模型的性能，以确保它正在学习。
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
# 训练过程需要经过多次迭代（ epochs ）。在每个时期，模型都会学习参数以做出更好的预测。
# 打印每个时期模型的准确性和损失；我们希望看到每个 epoch 的准确性都会提高，损失会减少。
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302227  [   64/60000]
loss: 2.290541  [ 6464/60000]
loss: 2.278565  [12864/60000]
loss: 2.271907  [19264/60000]
loss: 2.259622  [25664/60000]
loss: 2.240156  [32064/60000]
loss: 2.230141  [38464/60000]
loss: 2.207586  [44864/60000]
loss: 2.202543  [51264/60000]
loss: 2.174413  [57664/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 2.172049 

Epoch 2
-------------------------------
loss: 2.174152  [   64/60000]
loss: 2.166521  [ 6464/60000]
loss: 2.116733  [12864/60000]
loss: 2.127808  [19264/60000]
loss: 2.088179  [25664/60000]
loss: 2.039039  [32064/60000]
loss: 2.047876  [38464/60000]
loss: 1.980942  [44864/60000]
loss: 1.980299  [51264/60000]
loss: 1.912224  [57664/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.914990 

Epoch 3
-------------------------------
loss: 1.937685  [   64/60000]
loss: 1.912939  [ 6464/60000]
loss: 1.804067  [12864/60000]
loss: 1.839959  [19264/60000]
loss: 1.741695  [25664/60000]
loss: 1.691307  [32064/600

## 4.保存模型
保存模型的常见方法是序列化内部状态字典（包含模型参数）。

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 5.加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载到其中